1. 저장된 이미지들 비교해서 threshold 정하기


In [6]:
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

In [7]:
def matching(img1,img2,network): #img pair를 입력으로 받아서 유클라디안 거리 계산
    transform=transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()])
    
    img1 = Image.open(img1)
    img2 = Image.open(img2)
    # img1_ = img1.convert("L")
    # img2_ = img2.convert("L") #grayscale로 변환

    img1 = transform(img1)
    img2 = transform(img2)
    # img1 = torch.squeeze(img1)

    img1 = torch.unsqueeze(img1, 0)
    img2 = torch.unsqueeze(img2,0)

    output1, output2 = network(img1.cuda(), img2.cuda()) # Tensor
    
    output_vec1 = np.array(output1.cpu().detach().numpy())
    output_vec2 = np.array(output2.cpu().detach().numpy())
    
#     print(output_vec1) # np.array
#     print(output_vec2)
    
    euclidean_distance = np.sqrt(np.sum(np.square(np.subtract(output_vec1, output_vec2))))
#     euclidean_distance = F.pairwise_distance(output1, output2)
#     euclidean_distance = F.pairwise_distance(output_vec1, output_vec2)
    return euclidean_distance

In [9]:
import os
from itertools import combinations
import cv2
import torch


pth_file = './save_model/290_0.083.pth'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = torch.load(pth_file).to(device)
model.eval()

img_path = './0'
img_list = []
for files in os.listdir(img_path):
    file_path = os.path.join(img_path, files)
    img_list.append(file_path)
combi = list(combinations(img_list, 2))
total = 0
for img1, img2 in combi:
    distance = matching(img1, img2, model)
    print(f"{img1}-{img2} = {distance}")
    total += distance
avg = total / len(combi)
print(f"평균 : {avg}")


./0\frame1.png-./0\frame10.png = 0.5703622698783875
./0\frame1.png-./0\frame11.png = 0.5296507477760315
./0\frame1.png-./0\frame12.png = 0.4587993919849396
./0\frame1.png-./0\frame13.png = 0.36059632897377014
./0\frame1.png-./0\frame14.png = 0.30329179763793945
./0\frame1.png-./0\frame15.png = 0.24985793232917786
./0\frame1.png-./0\frame16.png = 0.2294924557209015
./0\frame1.png-./0\frame17.png = 0.33733856678009033
./0\frame1.png-./0\frame18.png = 0.44683948159217834
./0\frame1.png-./0\frame19.png = 0.6612650752067566
./0\frame1.png-./0\frame2.png = 0.10603486746549606
./0\frame1.png-./0\frame20.png = 1.0682913064956665
./0\frame1.png-./0\frame21.png = 0.5029549598693848
./0\frame1.png-./0\frame22.png = 0.5104404091835022
./0\frame1.png-./0\frame23.png = 0.3621164560317993
./0\frame1.png-./0\frame24.png = 0.21841199696063995
./0\frame1.png-./0\frame25.png = 0.12418997287750244
./0\frame1.png-./0\frame26.png = 0.08088196814060211
./0\frame1.png-./0\frame27.png = 0.22080561518669128
./0

In [12]:
def matching(img1, img2, network, use_cuda):
    transform=transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()])
    
    img1 = Image.open(img1)
    img2 = Image.open(img2)

    img1 = transform(img1)
    img2 = transform(img2)

    img1 = torch.unsqueeze(img1, 0)
    img2 = torch.unsqueeze(img2,0)

    if use_cuda:
        img1 = img1.cuda()
        img2 = img2.cuda()
    output1, output2 = network(img1, img2)
    
    output_vec1 = np.array(output1.cpu().detach().numpy())
    output_vec2 = np.array(output2.cpu().detach().numpy())

    euclidean_distance = np.sqrt(np.sum(np.square(np.subtract(output_vec1, output_vec2))))

    return euclidean_distance

In [18]:
import os
from itertools import combinations
import cv2
import torch


pth_file = './save_model/290_0.083.pth'
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
print(device)
model = torch.load(pth_file).to(device)
model.eval()

img_path = './0'
def img_combinations(img_path):
    img_list = []
    for files in os.listdir(img_path):
        file_path = os.path.join(img_path, files)
        img_list.append(file_path)
    return list(combinations(img_list, 2))

def calc_dist(img_lists):
    total = 0
    for img1, img2 in img_lists:
        distance = matching(img1, img2, model, use_cuda)
        total += distance
    avg = total / len(img_lists)
    return avg

img_lists = img_combinations(img_path)
print(len(img_lists))
avg = calc_dist(img_lists[:5])
print(avg)

cuda
2701
0.44454010725021365
